<a href="https://colab.research.google.com/github/elliemci/agents/blob/main/agentic_workflow_llamaidx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndex Agentic Workflows

Workflows are the building blocks of agent or a multi-agent system

In [ ]:
!pip install llama-index

In [ ]:
!pip install llama-index-llms-huggingface-api

In [ ]:
!pip install tavily-python

In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ColabNotebooks/AgentsCourse

Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/AgentsCourse


## Agentic Workflow with One Agent

### LLM

In [5]:
from google.colab import userdata
from huggingface_hub import login
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

hf_hub_token = userdata.get('huggingface_hub_access_token')
login(token=hf_hub_token)

tavily_api_key = userdata.get('TAVILY_API_KEY')

In [13]:
# instantiating an LLM
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

In [ ]:
# !pip install llama-index llama-index-llms-openai

# from llama_index.llms.OpenAI import OpenAI

# openai_api_key = userdata.get('OPENAI_API_KEY')
# llm = OpenAI(model="gpt-4o-mini", api_key=api_key)

The engine of any agent is the LLM that does the text processing and reasoning

### Tool

**Agents** are not using a traditional programing logic of define steps, but they are semi-autonomous, using a set of tools and having a goal, they will use these tool to find out more about the state of the world, take action in response to instructions and achive the goal. Tavity, a smart search engine, designed to be used by LLMs provides a web search tool to agents.<br><br>
**Tools** are regular python functions with
* name and description which are used by the LLM to understand what the tool does
* tool's metadata, input and output types are anotated for the LLM to "understand" how to use the tool
* `async` is used to make workflow more efficient

In [11]:
from tavily import AsyncTavilyClient

async def search_web(query: str) -> str:  # input and output type annotations
  """Useful for using the web to answer questions."""
  client = AsyncTavilyClient(api_key=tavily_api_key)
  return str(await client.search(query))

### AgentWorkflow

A system prompt defines what the agent does. Initializes an AgentWorkflow from a list of tools or functions.

If the LLM is a function calling model, the workflow will use the FunctionAgent, otherwise, it will use the ReActAgent.

In [15]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that answers questions. If you dont know the answer, you can search the web for information."
)

### Running the Agent

An AgentWorkflow expext to start with a question or prompt, user_msg, which is passed to the agent

In [16]:
response = await workflow.run(user_msg="What is the weather in Portland, OR?")
print(str(response))

The current weather in Portland, OR is partly cloudy with a temperature of 6.7°C (44.1°F). The wind speed is 2.2 mph (3.6 km/h) from the NNE, and the humidity is 76%.


### State

By default, AgenticWorkflow is stateless, to keep a memory of previous runs, Workflows need to mantain state within and between runs with a `Context`

In [17]:
from llama_index.core.workflow import Context

# instantiate a new Context passing in the workflow to properly configure the Context object
ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Ellie, nice to meet you.",
    ctx=ctx  # give the configured context to the workflow
    )
print(str(response))

Nice to meet you, Ellie! It's great to finally put a face to the name. How are you today?


In [18]:
# pass the same context to a second run
response = await workflow.run(
    user_msg="What is my name?",
    ctx=ctx
    )
print(str(response))

Your name is Ellie.


### Context

The context is an object of memory and it is serializable and can be saved to a database, file, etc and loaded back later. `JsonSerializer` uses `json.dumps` and `json.loads` to serialize and deserialize the context, while `JsonPickleSerializer` uses `pickle` and can be used for context with not serializable objects

In [20]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

# convert Context to dictionary obj
ctx_dict = ctx.to_dict(serializer=JsonSerializer())
# create a new Context from the dictionary
restored_ctx = Context.from_dict(workflow, ctx_dict, serializer=JsonSerializer())

response = await workflow.run(
    user_msg="Do you still remember my name?",
    ctx=restored_ctx
    )

print(str(response))

Yes, I still remember your name. Your name is Ellie.


### Streaming

`AgentWorkflow` can be streamed using a handler, which returns a variety of event types as the workflow executes
* `AgentStream` events stream the LLM output
* `AgentInput` events return the running agent
* `AgentOutput` events returns called tools and agent outputs
* `ToolCall` and `ToolCallResults` track tool calls and outputs

In [25]:
from llama_index.core.agent.workflow import AgentInput, AgentOutput, AgentStream, ToolCall, ToolCallResult

hangler = workflow.run(user_msg="What is the weather in Montreal?")

# streaming the response as each of the deltas are arriving
async for event in hangler.stream_events():
  if isinstance(event, AgentStream):       # handle AgentStream events
    print(event.delta, end="", flush=True) # delta is the latest chunk of text recived from the llm
    #print(event.response)  # the current full response
    #print(event.raw)       # the raw llm api response
    #print(event.current_agent_name)
 #elif isinstance(event, AgentInput):
    #print("Agent input:", event.input) # the current input message
    #print("Agent name:", event.current_agent_name)
 #elif isinstance(event, AgentOutput):
    #print("Agent output:", event.response)
    #print("Tool calls made:", event.tool_calls)
    #print("Raw LLM response:", event.raw)  # the raw llm api response
 #elif isinstance(event, ToolCall):
    #print("Tool called: ", event.tool_name)
    #print("Arguments to the tool:", event.tool_kwargs)
    #print("Tool output:", event.tool_output)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_web
Action Input: {"query": "current weather in Montreal"}Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The current weather in Montreal is overcast with a temperature of 0.2°C (32.4°F). It is currently day, and the wind is blowing at 6.3 mph (10.1 kph) from the WSW direction. The humidity is 74%, and the cloud cover is 100%.

### State Tools

Tools can be define to have access to the workflow context. The can set and retrive variables from the context. AgentWorkflow uses a context variable `state` that gets passed to every agent.

In [41]:
from llama_index.core.workflow import Context

# for the AgentWOrkflow to automatically pass the Context, the Context parameter should be the first input paramer of the tool
async def set_name(ctx: Context, name: str) -> str:
  state = await ctx.get("state")
  state["name"] = name
  await ctx.set("state", state)
  return f"Name set to {name}"

workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name":"unset"}
)

# initialize the context
ctx = Context(workflow)

# run the workflow
response = await workflow.run(user_msg="My name is Ellie", ctx=ctx)
print(str(response))

# access the state
state = await ctx.get("state")
print("Name as stored in state: ", state["name"])


Your name has been set to Ellie.
Name as stored in state:  Ellie


### Human in the Loop

The tool emits, `write_event_to_stream`,  an event that isn't recived by any other step in the workflow, the tool weaits till it receives a `InputRequiredEvent` or `HumanResponseEvent` events which can be subclassed to match the needs

In [11]:
from llama_index.core.workflow import Context, InputRequiredEvent, HumanResponseEvent
from llama_index.core.agent.workflow import AgentWorkflow

# a tool that [erforms a dangerous task
async def dangerous_task(ctx: Context) -> str:
  """A dangerous task that requires human confirmation."""

  # emit an event to the external stream to be captured
  ctx.write_event_to_stream(
      InputRequiredEvent(
          prefix="Are you sure you want to proceed?",
          user_name="Ellie"
      )
  )

  # wait untill a HumanResposeEvent
  response = await ctx.wait_for_event(
      HumanResponseEvent, requirements={"user_name": "Ellie"}
  )

  # act on the input from the event
  if response.response.stri().lower() == "yes":
    return "Dangerous task completed successfully."
  else:
    return "Dangerous task aborted."

To capture the event, use the streaming interface with `InputRequiredEvent` to capture a response from the user and sent it back using `send_event` method

In [17]:
workflow = AgentWorkflow.from_tools_or_functions(
      [dangerous_task],
      llm=llm,
      system_prompt="You are a helpful assistant that can perform a dangerous task."
  )

handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

async for event in handler.stream_events():
  # capture InputRequiredEvent
  if isinstance(event, InputRequiredEvent):
    # capture keyboard input
    response = input(event.prefix)
    # send our response back
    handler.ctx.send_event(
        HumanResponseEvent(
            response=response,
            user_name=event.user_name
        )
      )

response = await handler
print(str(response))

Are you sure you want to proceed?Yes
The dangerous task has been confirmed and is now being executed. Please proceed with the necessary precautions.
